<a href="https://colab.research.google.com/github/mlraul/biosensor_predictor/blob/main/Model_validation_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Net training whit K-Fold Cross Validation
In this code, the net will be validated using the K-Fold CV method.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Activation, Dropout, Input
from keras.layers.merge import Concatenate
from keras import metrics
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import classification_report

###Loading inputs

- fp_m is a matrix where each row contains the fingerprint of a molecule. Each fingerprint is written in two successive rows.
- aa_m is a 3-Dimensional array where each matrix represents the amino acids of the Transcription Factor (TF).

In [ ]:
# PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1EeFJCysQ6Ts0ebDkxMuohrRTYo4T0GDI"})
downloaded.GetContentFile('aa_neg_no_motifs_matrix.npy')
aa_m = np.load('aa_neg_no_motifs_matrix.npy')

downloaded = drive.CreateFile({'id':"1co-dTn5YYPiXKgYTFJ6-wi2wGVoHXF7G"})
downloaded.GetContentFile('fingerprints_v3_matrix.npy')
fp_m = np.load('fingerprints_v3_matrix.npy')

###Creating the labels
In our current input, every amino acids sequence is given twice:

- The first one is associated to the fingerprint which represents the molecule with higher affinity with this TF.
-The second one is associated to a random fingerprint to which the affinity is lower.

With this idea in mind, the labels set consists on a repeated sequence of 1 and 0 (with 1 meaning affinity between the TF and the molecule, and 0 the opposite).

In [ ]:
lbl = np.ones(len(fp_m))

for i in range(1, len(lbl), 2):
  lbl[i] = 0

###Splitting the data
Here two data subsets will be created:

- The train data, which will be used to train the net. This will be splitted again in different folds when using the K-F CV.
- The test data, which will be used to evaluate it.

In [ ]:
aa_train, aa_test, fp_train, fp_test, lbl_train, lbl_test = train_test_split(aa_m, fp_m, lbl, test_size = 0.20)

###Model configuration
Here different parameters of the net architecture will be defined.

In [ ]:
# Input layers shape
i1_shape = np.shape(aa_m)[1:]
i2_shape = np.shape(fp_m)[1]

# Model parameters
loss_function = 'binary_crossentropy'
opt = Adam(learning_rate=0.0001)
metrics = ['accuracy']

num_folds = 5
batch_size = 50
num_epochs = 30 

lstm_units = 90
densefp_units = 70
dense_units = 70

verbosity = 0

###K-Fold definition
It defines de K-Fold Cross Validator.

In [ ]:
kfold = KFold(n_splits = num_folds)

### Creating the net architecture
It has two branches:
- One for the amino acids and motifs matrixes. Here a LSTM layer is used, as the inputs are actually sequences.
- Other for the molecules fingerprints. By the moment, a Dense layer is used here.

Then, these two branches are concatenated. The current architecture finishes with a Dense layer as output.

In [ ]:
# Branch for sequences matrixes. A LSTM layer is used
input1 = Input(shape = i1_shape)
i11 = LSTM(lstm_units, activation='relu', recurrent_activation='sigmoid',
           dropout=0.2, recurrent_dropout=0.01)(input1)

# Branch for fingerprints
input2 = Input(shape = i2_shape)
i21 = Dense(densefp_units, activation='relu')(input2)

# Joint of branches
concat = Concatenate()([i11, i21])

# Other layers TEST (DROPOUTS)
dense1 = Dense(dense_units, activation='relu')(concat)

# Output layer
out = Dense(1)(dense1)

# Creation and compilation of the model
model = Model(inputs = [input1, input2], outputs = out)
model.compile(loss = loss_function, optimizer = opt, metrics = metrics)
print(model.summary())


###K-Fold CV model evaluation
Here the CV evaluation will be performed. 

In [ ]:
# Defining the per-fold score lists
acc_per_fold = []
loss_per_fold = []

fold_num = 1

# Saving the initial model weights
Wsave = model.get_weights()
    
for train, val in kfold.split(aa_train, fp_train, lbl_train):

  model.set_weights(Wsave)
    
  # Generate a print
  print('------------------------------------------------------------------')
  print(f'Training for fold {fold_num} ...')
    
    
  history = model.fit([aa_train[train], fp_train[train]], lbl_train[train],
                      validation_data=([aa_train[val], fp_train[val]], lbl_train[val]),
                      batch_size = batch_size, 
                      epochs = num_epochs, 
                      verbose = verbosity)
    
  # summarize history for accuracy
  plt.figure(dpi=100)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epochs')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

  # summarize history for loss
  plt.figure(dpi=100)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epochs')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

  fold_loss = history.history['val_loss'][-1]
  fold_accuracy = history.history['val_accuracy'][-1] * 100
  print(f'Score for fold {fold_num}: {list(history.history.keys())[2]} of {fold_loss}; {list(history.history.keys())[3]} of {fold_accuracy} %')

  acc_per_fold.append(fold_accuracy)
  loss_per_fold.append(fold_loss)
    
  # Increasing fold number
  fold_num = fold_num + 1

###Average scores
Here the score for each fold will be displayed, as well as the average of all scores. 
This result will be used to evaluate how well the model is performing.

In [ ]:
print('------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} %')
print('------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} % (+- {np.std(acc_per_fold)} %)')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------')

###Training the selected model
If the model validated is decided to be the definitive model, the final training will be carried out using all the training data. For this purpose, type 'train' when asking for it. Otherwise, the model will not be trained.

If the model is decided to be trained, the test will also be applied. The curve ROC is plotted and a classification report is printed.

In [ ]:
print('Is the model validated prepared to be trained?')
answer = input("Press 'train' for training the model, press any other key for not training it: ")
if answer == 'train':
  print('Starting training...')

  model.set_weights(Wsave)

  history = model.fit([aa_train, fp_train], lbl_train,
                      batch_size = batch_size, 
                      epochs = num_epochs, 
                      verbose = verbosity)

  loss = history.history['loss'][-1]
  accuracy = history.history['accuracy'][-1] * 100
  print('------------------------------------------------------------------')
  print('Scores:')
  print(f'> Accuracy: {accuracy} %')
  print(f'> Loss: {loss}')
  print('------------------------------------------------------------------')

  results = model.evaluate([aa_test, fp_test], lbl_test, 
                            verbose=0)
  print(f'{model.metrics_names[0]} of {results[0]}; {model.metrics_names[1]} of {results[1]*100} %')

  preds_test = model.predict([aa_test, fp_test]).ravel()
  fpr, tpr, thresholds = roc_curve(lbl_test, preds_test)
  auc_value = auc(fpr, tpr)

  plt.figure(dpi=100)
  plt.plot([0, 1], [0, 1], 'k--')
  plt.plot(fpr, tpr, label='AUC = {:.3f})'.format(auc_value))
  plt.xlabel('False positive rate')
  plt.ylabel('True positive rate')
  plt.title('ROC curve')
  plt.legend(loc='best')
  plt.show()

  preds_binary = preds_test.copy()
  for i in range(len(preds_binary)):
    if preds_binary[i] > 0.5:
      preds_binary[i] = 1
    else:
      preds_binary[i] = 0

  print(classification_report(lbl_test, preds_binary))

else:
  print('Model discarded')

###Saving the model
Once the model is fully trained and tested, it is saved in order to be used to make future predictions. For saving it type 'save' when asking for it. Otherwise the model will not be saved.

In [ ]:
print('Do you want to save the model?')
answer = input("Press 'save' for saving the model, press any other key for not saving it: ")
if answer == 'save':
  print('Saving model...')
  # Saves the entire model as a SavedModel.
  !mkdir -p saved_model
  model.save('saved_model/final_model')

else:
  print('Model not saved')
